In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE55457"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE55457"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE55457.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE55457.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE55457.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of rheumatoid arthritis and osteoarthritis patients by transcriptome-based rule set generation [Jena]"
!Series_summary	"Discrimination of rheumatoid arthritis (RA) patients from patients with other inflammatory/degenerative joint diseases or healthy individuals purely on the basis of genes differentially expressed in high-throughput data has proven very difficult. Thus, the present study sought to achieve such discrimination by employing a novel unbiased approach using rule-based classifiers.  Three multi-center genome-wide transcriptomic data sets (Affymetrix HG- U133 A/B) from a total of 79 individuals, including 20 healthy controls (control group - CG), as well as 26 osteoarthritis (OA) and 33 RA patients, were used to infer rule- based classifiers to discriminate the disease groups. The rules were ranked with respect to Kiendl’s statistical relevance index, and the resulting rule set was optimized by pruning. The rule sets were 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains genome-wide transcriptomic data from Affymetrix HG-U133 A/B
# which means it contains gene expression data
is_gene_available = True

# 2.1 Data Availability
# Examining the sample characteristics dictionary:
# - trait: in key 2 as 'clinical status', which includes 'osteoarthritis' (our trait of interest)
# - age: in key 1 as age
# - gender: in key 0 as gender
trait_row = 2
age_row = 1
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (1 for osteoarthritis, 0 for others)"""
    if value is None:
        return None
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Return 1 for osteoarthritis, 0 for others
    if 'osteoarthritis' in value:
        return 1
    elif 'normal control' in value or 'rheumatoid arthritis' in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numerical value"""
    if value is None:
        return None
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Load the sample characteristics file from the previous step
    # We're assuming clinical_data is already loaded from previous steps
    # Or that it will be provided to us in the execution environment
    try:
        # First try loading a pre-existing clinical_data variable
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the DataFrame
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the DataFrame to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("Clinical data processing failed. Check if clinical_data is properly loaded.")


Preview of selected clinical features:
{'GSM1337304': [0.0, 61.0, 1.0], 'GSM1337305': [0.0, 64.0, 1.0], 'GSM1337306': [0.0, 78.0, 0.0], 'GSM1337307': [0.0, 65.0, 1.0], 'GSM1337308': [0.0, 53.0, 1.0], 'GSM1337309': [0.0, 68.0, 1.0], 'GSM1337310': [0.0, 29.0, 0.0], 'GSM1337311': [0.0, 17.0, 1.0], 'GSM1337312': [0.0, 39.0, 1.0], 'GSM1337313': [0.0, 36.0, 1.0], 'GSM1337314': [0.0, 75.0, 1.0], 'GSM1337315': [0.0, 79.0, 0.0], 'GSM1337316': [0.0, 63.0, 0.0], 'GSM1337317': [0.0, 66.0, 0.0], 'GSM1337318': [0.0, 64.0, 0.0], 'GSM1337319': [0.0, 63.0, 0.0], 'GSM1337320': [0.0, 46.0, 0.0], 'GSM1337321': [0.0, 71.0, 0.0], 'GSM1337322': [0.0, 72.0, 0.0], 'GSM1337323': [0.0, 2.0, 0.0], 'GSM1337324': [0.0, 47.0, 1.0], 'GSM1337325': [0.0, 59.0, 1.0], 'GSM1337326': [0.0, 73.0, 0.0], 'GSM1337327': [1.0, 77.0, 0.0], 'GSM1337328': [1.0, 71.0, 0.0], 'GSM1337329': [1.0, 76.0, 0.0], 'GSM1337330': [1.0, 61.0, 0.0], 'GSM1337331': [1.0, 75.0, 0.0], 'GSM1337332': [1.0, 78.0, 1.0], 'GSM1337333': [1.0, 69.0, 1.0], '

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE55457/GSE55457_series_matrix.txt.gz
Gene data shape: (22283, 33)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed (e.g., '1007_s_at', '1053_at'), these appear to be 
# Affymetrix probe IDs rather than human gene symbols.
# Affymetrix IDs need to be mapped to standard gene symbols for consistent analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the columns for gene identifiers and gene symbols in the annotation dataframe
# Based on the preview, we can see:
# - 'ID' column contains probe IDs (e.g., '1007_s_at') matching the gene expression data index
# - 'Gene Symbol' column contains gene symbols (e.g., 'DDR1 /// MIR4640')
probe_col = 'ID'
gene_col = 'Gene Symbol'

print(f"Using {probe_col} as probe identifier column and {gene_col} as gene symbol column")

# 2. Get mapping dataframe from gene annotation
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First 5 rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols in the mapped gene expression data:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to handle duplicates and synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First 10 gene symbols after normalization:")
print(gene_data.index[:10])

# 5. Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using ID as probe identifier column and Gene Symbol as gene symbol column
Gene mapping dataframe shape: (21225, 2)
First 5 rows of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Gene expression data shape after mapping: (13830, 33)
First 10 gene symbols in the mapped gene expression data:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')
Gene expression data shape after normalization: (13542, 33)
First 10 gene symbols after normalization:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE55457.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (13542, 33)
Gene data column names (sample IDs):
Index(['GSM1337304', 'GSM1337305', 'GSM1337306', 'GSM1337307', 'GSM1337308'], dtype='object')

Raw clinical data structure:
Clinical data shape: (3, 34)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM1337304', 'GSM1337305', 'GSM1337306',
       'GSM1337307'],
      dtype='object')

Sample characteristics dictionary:
{0: ['gender: male', 'gender: female'], 1: ['age: 61', 'age: 64', 'age: 78', 'age: 65', 'age: 53', 'age: 68', 'age: 29', 'age: 17', 'age: 39', 'age: 36', 'age: 75', 'age: 79', 'age: 63', 'age: 66', 'age: 46', 'age: 71', 'age: 72', 'age: 2', 'age: 47', 'age: 59', 'age: 73', 'age: 77', 'age: 76', 'age: 69', 'age: 80'], 2: ['clinical status: normal control', 'clinical status: rheumatoid arthritis', 'clinical status: osteoarthritis']}

Values in trait row:
['!Sample_characteristics_ch1' 'clinical status: normal control'
 'clinical status: normal control' 'clinical status: normal control'
 '

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (33, 13543)
For the feature 'Osteoarthritis', the least common label is '1' with 10 occurrences. This represents 30.30% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoarthritis/GSE55457.csv
